In [2]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
import numpy as np
from operator import itemgetter

import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.sparse as sps

In [3]:
from Data_Handler.DataReader import DataReader 
dataReader=DataReader()
URM_all=dataReader.load_augmented_binary_urm()
powerful_urm=dataReader.load_powerful_binary_urm()

#URM_train, URM_validation = split_train_in_two_percentage_global_sample(urm, train_percentage = 0.9)
#train_powerful, val_powerful = split_train_in_two_percentage_global_sample(powerful_urm, train_percentage = 0.9)

In [43]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

(41629, 24507)
(41629, 24507)
EvaluatorHoldout: Ignoring 291 ( 0.7%) Users that have less than 1 test interactions


In [39]:
MAP=[]

for topK in [10, 50, 100, 200, 500]:
    for shrink in [0, 10, 50, 100, 200, 500]:
        recommender = ItemKNNCFRecommender(URM_train)
        recommender.fit(topK=topK, shrink=shrink)

        result_dict, _ = evaluator_validation.evaluateRecommender(recommender)
        result_dict = result_dict.to_dict()
        print(result_dict)
        print(type(result_dict))
        MAP.append(('topK={}'.format(topK),'shrink={}'.format(shrink),result_dict["MAP"]))
        print(MAP)

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 24507 (100.0%), 1449.70 column/sec. Elapsed time 16.90 sec
EvaluatorHoldout: Processed 41293 (100.0%) in 12.39 sec. Users per second: 3332
{'PRECISION': {10: 0.05877509505242034}, 'PRECISION_RECALL_MIN_DEN': {10: 0.07953087094587757}, 'RECALL': {10: 0.068724601448325}, 'MAP': {10: 0.029331034046662496}, 'MAP_MIN_DEN': {10: 0.03887849147252289}, 'MRR': {10: 0.18357680247891364}, 'NDCG': {10: 0.08212248806781493}, 'F1': {10: 0.06336164074777062}, 'HIT_RATE': {10: 0.36473494296854186}, 'ARHR_ALL_HITS': {10: 0.2296895511326569}, 'NOVELTY': {10: 0.004558569894615962}, 'AVERAGE_POPULARITY': {10: 0.15186199239700923}, 'DIVERSITY_MEAN_INTER_LIST': {10: 0.9624897836779019}, 'DIVERSITY_HERFINDAHL': {10: 0.9962466474889883}, 'COVERAGE_ITEM': {10: 0.45729791488146243}, 'COVERAGE_ITEM_HIT': {10: 0.0721834577875709}, 'ITEMS_IN_GT': {10: 0.998327008609785}, 'COVERAGE_USER': {10: 0.9919287035480074}, 'COVERAGE_USER_HIT': {

KeyboardInterrupt: 

In [34]:
MAP

['topK=10',
 'shrink=0',
 0.029331034046662496,
 'topK=10',
 'shrink=10',
 0.029323209591231945,
 'topK=10',
 'shrink=50',
 0.029311150973356393,
 'topK=10',
 'shrink=100',
 0.029297173240092123,
 'topK=10',
 'shrink=200',
 0.029306732299066364,
 'topK=10',
 'shrink=500',
 0.029275828487013125,
 'topK=50',
 'shrink=0',
 0.0309715077193209,
 'topK=50',
 'shrink=10',
 0.03097090228982921,
 'topK=50',
 'shrink=50',
 0.030967871298373986,
 'topK=50',
 'shrink=100',
 0.03096935508112821,
 'topK=50',
 'shrink=200',
 0.03094246344070651,
 'topK=50',
 'shrink=500',
 0.030922445827513335,
 'topK=100',
 'shrink=0',
 0.031228393374639618,
 'topK=100',
 'shrink=10',
 0.031221891254098806,
 'topK=100',
 'shrink=50',
 0.031210596630581736,
 'topK=100',
 'shrink=100',
 0.031197507821571144,
 'topK=100',
 'shrink=200',
 0.031215748547256134,
 'topK=100',
 'shrink=500',
 0.031150191104298265,
 'topK=200',
 'shrink=0',
 0.03089011685365652,
 'topK=200',
 'shrink=10',
 0.030884634353259624,
 'topK=200',


In [28]:
max_tuple = max(MAP, key=itemgetter(2))
print('The best combination for the ICF recommender is ({},{}), with a MAP = {}'.format(max_tuple[0],max_tuple[1], max_tuple[2]))

TypeError: 'str' object is not callable

In [6]:
MAP=[]

for num_factors in np.logspace(1,15,10, base=2).astype(int):
    recommender = PureSVDRecommender(URM_train)
    recommender.fit(num_factors = num_factors)

    result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

    MAP.append(('number of factors={}'.format(num_factors),result_dict[10]["MAP"]))

PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 0.27 sec
EvaluatorHoldout: Processed 41311 (100.0%) in 22.34 sec. Users per second: 1849


KeyError: 10